Environment

In [ ]:
import os
import sys
import importlib
if (importlib.util.find_spec("ronek") is None):
  sys.path.append("./../../../")

from ronek import env
env.set(
  device="cpu",
  device_idx=0,
  nb_threads=8,
  floatx="float64"
)

Import libraries

In [ ]:
import numpy as np

from utils import *
from ronek import const
from ronek.systems import TAFASystem
from silx.io.dictdump import h5todict

Define inputs

In [ ]:
# System
T = 1e4
ic = {
  "cold": [5e2, 1e3, 5e-2],
  "hot":  [3e3, 1e3, 5e-2]
}
tgrid = {
  "lim": [1e-9, 1e-3],
  "num": 501
}
teval = {
  "cold": [1e-8, 1e-7, 7e-7, 3e-6, 1e-4],
  "hot":  [1e-7, 1e-6, 4e-6, 2e-5, 1e-3]
}
# rom_dims = list(range(8,15,2))
rom_dims = list(range(4,7))
# Paths
paths = {
  "dtb": "./../database/",
  "data": "./data/max_mom_2/"
}
# Plotting distribution
pltt_dist = False

Initialize isothermal master equation model

In [ ]:
model = TAFASystem(
  rates=paths["dtb"] + "/kinetics.hdf5",
  species={
    k: paths["dtb"] + f"/species/{k}.json" for k in ("atom", "molecule")
  },
  use_einsum=False
)
model.update_fom_ops(T)

> Molecule's levels

In [ ]:
gi = model.species["molecule"].lev["g"]
ei = model.species["molecule"].lev['e'] / const.eV_to_J

Balanced POD

In [ ]:
os.makedirs(paths["data"]+"/figs/bases/", exist_ok=True)

In [ ]:
bases = h5todict(paths["data"]+"/bases.hdf5")
s, phi, psi = [bases[k] for k in ("s", "phi", "psi")]

In [ ]:
# Cumulative energy
cs = 1.0 / np.sum(s**2)
cs *= np.cumsum(s**2)
# Number of principal components
eps = 1e-5
rom_dim = np.where(cs > 1-eps)[0][0]+1
rom_dim

In [ ]:
pltt.cum_energy(
  cs[:rom_dims[-1]],
  figname=paths["data"]+"/figs/bases/cum_en.png",
  save=True,
  show=False
)

In [ ]:
for i in range(rom_dims[-1]):
  nb = str(i+1)
  for (name, basis) in (("phi", phi), ("psi", psi)):
    pltt.dist_2d(
      x=ei,
      y=basis[:,i],
      labels=[r"$\epsilon_i$ [eV]", r"$\%s_{%s}$" % (name, nb)],
      scales=["linear", "linear"],
      figname=paths["data"] + f"/figs/bases/{name}_{nb.zfill(2)}.png",
      save=True,
      show=False
    )

FOM and ROM solutions

In [ ]:
t = get_tgrid(tgrid["lim"], tgrid["num"])
for (name, (T, p, Xa)) in ic.items():
  y0 = get_y0(model, T, p, Xa)
  # Solving
  yfom = solve_fom(model, t, y0)
  for r in rom_dims:
    yrom = solve_rom(model, t, y0, phi, psi, r, abs=False)
    # Postprocessing
    path = paths["data"]+f"/figs/sol/{name}_r{r}/"
    os.makedirs(path, exist_ok=True)
    # > Moments
    plot_moments(path, t, yfom[1], yrom[1], ei.reshape(-1,1), max_mom=5)
    if pltt_dist:
      # > Distribution static
      plot_dist(path, teval[name], t, yfom[1], yrom[1], ei, gi)
      # > Distribution dynamic
      animate_dist(path, t, yfom[1], yrom[1], ei, gi)